In [ ]:
import numpy as np
import cv2
def auto_canny(image, sigma=0.33):
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edges = cv2.Canny(image, lower, upper)
    return edges


In [ ]:
def density_measure(image):
    edges_ref = cv2.Canny(image, 100, 200)
    #returns percentage of edge pixels in the image.
    return np.mean(edges_ref > 0)

    

In [ ]:

def choose_params(dens):
    params = {
        "clahe_clip": 2.0,
        "clahe_tile": (8, 8),
        "bilateral_d": 7,
        "bilateral_sigmaColor": 50,
        "bilateral_sigmaSpace": 50,
        "canny_sigma": 0.25
    }

    if dens > 0.12:   
        params["clahe_clip"] = 1.5
        params["bilateral_d"] = 9
        params["bilateral_sigmaColor"] = 75
        params["bilateral_sigmaSpace"] = 75
        params["canny_sigma"] = 0.18
    elif dens < 0.03:
        params["clahe_clip"] = 2.5
        params["bilateral_d"] = 5
        params["bilateral_sigmaColor"] = 40
        params["bilateral_sigmaSpace"] = 40
        params["canny_sigma"] = 0.30

    return params




In [ ]:
import matplotlib.pyplot as plt
se_rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
from skimage.morphology import skeletonize


def enhancementPipeline(imgUrl):
    img = cv2.imread(imgUrl)
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    density = density_measure(grayImg)
    params = choose_params(density)
    clahe = cv2.createCLAHE(params["clahe_clip"], tileGridSize=(8, 8))
    enhanced = clahe.apply(grayImg)
    blur = cv2.bilateralFilter(enhanced,params["bilateral_d"], params["bilateral_sigmaColor"], params["bilateral_sigmaSpace"])
    edges = auto_canny(blur ,params["canny_sigma"])
    fig, (ax1, ax2 , ax3) = plt.subplots(1, 3, figsize=(10, 5))
    ax1.imshow(img, cmap="gray")
    ax2.imshow(blur, cmap="gray")
    ax3.imshow(edges, cmap="gray")

In [ ]:
import glob
import os
folder_path = "puzzle_pieces_2x2"
print("Folder exists:", os.path.exists(folder_path))  

image_paths = glob.glob(os.path.join(folder_path, "*.jpg")) \
             + glob.glob(os.path.join(folder_path, "*.png")) \
             + glob.glob(os.path.join(folder_path, "*.jpeg"))
enhancementPipeline(image_paths[47])